In [1]:
import os
import zipfile
import pandas as pd

ZIP_PATH = r"C:\Users\avi04\Downloads\air_quality_dataset (1).zip"
EXTRACT_DIR = r"C:\Users\avi04\Downloads\air_quality_dataset"
INPUT_FILE = os.path.join(EXTRACT_DIR, "city_day.csv")
OUTPUT_FILE = r"C:\Users\avi04\Downloads\city_day_cleaned.csv"

POLLUTANTS = ['PM2.5','PM10','NO','NO2','NOx','NH3','CO','SO2','O3']
VOCs = ['Benzene','Toluene','Xylene']
GLOBAL_FALLBACK_COLS = ['PM10','NOx','NH3','O3']

def extract_zip(zip_path, extract_dir):
    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_dir)

def clean_city_day(df):
    df = df.copy()
    df["Date"] = pd.to_datetime(df["Date"], errors='coerce')
    for col in POLLUTANTS + VOCs:
        if col in df.columns:
            df[col] = df.groupby("City")[col].transform(lambda x: x.fillna(x.median()))
    for col in GLOBAL_FALLBACK_COLS:
        if col in df.columns:
            df[col] = df[col].fillna(df[col].median())
    df = df.drop(columns=["AQI","AQI_Bucket"], errors="ignore")
    cols = df.columns.tolist()
    ordered = ["City","Date"] + [c for c in cols if c not in ["City","Date"]]
    return df[ordered]

def main():
    extract_zip(ZIP_PATH, EXTRACT_DIR)
    df = pd.read_csv(INPUT_FILE)
    cleaned = clean_city_day(df)
    cleaned.to_csv(OUTPUT_FILE, index=False)
    print("Saved cleaned file to:", OUTPUT_FILE)

if __name__ == "__main__":
    main()


Saved cleaned file to: C:\Users\avi04\Downloads\city_day_cleaned.csv


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\avi04\Downloads\city_day_cleaned.csv")

def compute_subindex(C, breakpoints):
    for BPlo, BPhi, Ilo, Ihi in breakpoints:
        if BPlo <= C <= BPhi:
            return ((Ihi - Ilo) / (BPhi - BPlo)) * (C - BPlo) + Ilo
    return np.nan

pm25_bp = [
    (0, 30, 0, 50),
    (31, 60, 51, 100),
    (61, 90, 101, 200),
    (91, 120, 201, 300),
    (121, 250, 301, 400),
    (251, 500, 401, 500)
]

pm10_bp = [
    (0, 50, 0, 50),
    (51, 100, 51, 100),
    (101, 250, 101, 200),
    (251, 350, 201, 300),
    (351, 430, 301, 400),
    (430, 1000, 401, 500)
]

no2_bp = [
    (0, 40, 0, 50),
    (41, 80, 51, 100),
    (81, 180, 101, 200),
    (181, 280, 201, 300),
    (281, 400, 301, 400),
    (401, 1000, 401, 500)
]

so2_bp = [
    (0, 40, 0, 50),
    (41, 80, 51, 100),
    (81, 380, 101, 200),
    (381, 800, 201, 300),
    (801, 1600, 301, 400),
    (1601, 2000, 401, 500)
]

co_bp = [
    (0, 1, 0, 50),
    (1, 2, 51, 100),
    (2, 10, 101, 200),
    (10, 17, 201, 300),
    (17, 34, 301, 400),
    (34, 50, 401, 500)
]

o3_bp = [
    (0, 50, 0, 50),
    (51, 100, 51, 100),
    (101, 168, 101, 200),
    (169, 208, 201, 300),
    (209, 748, 301, 400),
    (749, 1000, 401, 500)
]

df["SI_PM25"] = df["PM2.5"].apply(lambda x: compute_subindex(x, pm25_bp))
df["SI_PM10"] = df["PM10"].apply(lambda x: compute_subindex(x, pm10_bp))
df["SI_NO2"] = df["NO2"].apply(lambda x: compute_subindex(x, no2_bp))
df["SI_SO2"] = df["SO2"].apply(lambda x: compute_subindex(x, so2_bp))
df["SI_CO"] = df["CO"].apply(lambda x: compute_subindex(x, co_bp))
df["SI_O3"] = df["O3"].apply(lambda x: compute_subindex(x, o3_bp))

df["AQI"] = df[["SI_PM25","SI_PM10","SI_NO2","SI_SO2","SI_CO","SI_O3"]].max(axis=1)

def bucket(aqi):
    if aqi <= 50: return "Good"
    elif aqi <= 100: return "Satisfactory"
    elif aqi <= 200: return "Moderate"
    elif aqi <= 300: return "Poor"
    elif aqi <= 400: return "Very Poor"
    else: return "Severe"

df["AQI_Bucket"] = df["AQI"].apply(bucket)

df.to_csv(r"C:\Users\avi04\Downloads\city_day_with_AQI.csv", index=False)

print("AQI calculation complete!")


AQI calculation complete!


In [3]:
import pandas as pd

df = pd.read_csv(r"C:\Users\avi04\Downloads\city_day_with_AQI.csv")

pollutants = ["PM2.5","PM10","NO","NO2","NOx","NH3","CO","SO2","O3"]

# Basic descriptive statistics
desc_stats = df[pollutants].agg(["mean", "median", "min", "max"]).T

# Mode for each pollutant
mode_stats = df[pollutants].mode().iloc[0]

desc_stats["mode"] = mode_stats

print("Descriptive Statistics:\n")
print(desc_stats)

desc_stats.to_csv(r"C:\Users\avi04\Downloads\descriptive_statistics.csv")


Descriptive Statistics:

             mean   median   min      max     mode
PM2.5   64.495018   48.170  0.04   949.99   23.745
PM10   111.529814  100.685  0.01  1000.00  100.685
NO      17.586426   10.580  0.02   390.68   25.340
NO2     28.038496   21.460  0.01   362.21   21.460
NOx     31.738348   24.290  0.00   467.63   24.290
NH3     21.785599   14.500  0.01   352.89   14.500
CO       2.469716    0.900  0.00   175.81    0.000
SO2     15.120249    9.900  0.01   193.86   12.670
O3      33.794051   29.620  0.01   257.73   29.620


In [4]:
subindex_cols = ["SI_PM25","SI_PM10","SI_NO2","SI_SO2","SI_CO","SI_O3"]

def dominant_pollutant(row):
    return row[subindex_cols].idxmax().replace("SI_", "")

df["Dominant_Pollutant"] = df.apply(dominant_pollutant, axis=1)

city_dom = df.groupby("City")["Dominant_Pollutant"].agg(lambda x: x.value_counts().index[0])

city_dom.to_csv(r"C:\Users\avi04\Downloads\dominant_pollutant_city.csv")

print("\nDominant pollutant per city:\n")
print(city_dom)



Dominant pollutant per city:

City
Ahmedabad               CO
Aizawl                PM25
Amaravati             PM10
Amritsar              PM10
Bengaluru             PM10
Bhopal                PM10
Brajrajnagar          PM10
Chandigarh            PM10
Chennai               PM25
Coimbatore              CO
Delhi                 PM25
Ernakulam               CO
Gurugram              PM25
Guwahati              PM25
Hyderabad             PM10
Jaipur                PM10
Jorapokhar            PM10
Kochi                   CO
Kolkata               PM10
Lucknow               PM25
Mumbai                PM10
Patna                 PM25
Shillong              PM25
Talcher               PM10
Thiruvananthapuram      CO
Visakhapatnam         PM10
Name: Dominant_Pollutant, dtype: object


In [7]:
import pandas as pd
import numpy as np

path = r"C:\Users\avi04\Downloads\city_day_cleaned.csv"
df = pd.read_csv(path)

# Try parsing robustly: first try dayfirst, then fall back with errors='coerce'
# Use parse_dates on the column if you prefer; here we attempt two passes for robustness.
def parse_dates_try(col):
    # 1) try fast parse with dayfirst
    parsed = pd.to_datetime(col, dayfirst=True, errors='coerce')
    # 2) where parse failed, try without dayfirst (ISO-ish)
    mask_failed = parsed.isna()
    if mask_failed.any():
        parsed2 = pd.to_datetime(col[mask_failed], dayfirst=False, errors='coerce')
        parsed.loc[mask_failed] = parsed2
    return parsed

df['Date_parsed'] = parse_dates_try(df['Date'])

# Report failures
n_failed = df['Date_parsed'].isna().sum()
print(f"Total rows: {len(df)}, Unparsed date rows: {n_failed}")

if n_failed > 0:
    print("Sample of rows with unparsed Date values:")
    display(df.loc[df['Date_parsed'].isna(), ['Date']].drop_duplicates().head(20))

# If you prefer to see their indices/rows:
# display(df[df['Date_parsed'].isna()].head(10))

# If many are NaT you can decide to drop or inspect. For now we'll keep and create Year/Month with NaNs handled.
df['Date'] = df['Date_parsed']  # replace/overwrite original
df = df.drop(columns=['Date_parsed'])

# Create Year and Month columns (will be NaN for unparsed dates)
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

# Save a quick version for further work
out_path = r"C:\Users\avi04\Downloads\city_day_cleaned_dates_fixed.csv"
df.to_csv(out_path, index=False)
print("Saved parsed dataset to:", out_path)


Total rows: 29531, Unparsed date rows: 0
Saved parsed dataset to: C:\Users\avi04\Downloads\city_day_cleaned_dates_fixed.csv


In [8]:
df["Year"] = pd.to_datetime(df["Date"]).dt.year


In [9]:
city_year_avg = df.groupby(["City","Year"])[pollutants].mean()

city_year_avg.to_csv(r"C:\Users\avi04\Downloads\city_yearly_pollutant_avg.csv")

print(city_year_avg)


                        PM2.5        PM10         NO        NO2        NOx  \
City          Year                                                           
Ahmedabad     2015  74.339918  107.960000  14.032000  25.446493  34.286164   
              2016  59.780929  107.960000  15.804973  36.183470  35.454044   
              2017  64.447288  107.960000  18.861205  52.753836  43.790712   
              2018  74.599370  107.960000  33.151452  84.730658  60.317342   
              2019  62.056849  115.538849  25.971863  90.369205  62.721123   
...                       ...         ...        ...        ...        ...   
Visakhapatnam 2016  44.697500   88.346630  16.461902  42.183261  32.574022   
              2017  48.448767  104.437849  11.770301  34.602630  12.314521   
              2018  49.894822  115.826808  12.341370  38.764466  30.248082   
              2019  47.156055  114.403014  13.863370  37.672904  31.185288   
              2020  32.104645   83.325355   6.679454  30.957432 

In [12]:
import pandas as pd
import numpy as np

path = r"C:\Users\avi04\Downloads\city_day_cleaned.csv"
df = pd.read_csv(path, dtype=str)

def try_parse_dates(s):
    p = pd.to_datetime(s, dayfirst=True, errors='coerce')
    mask = p.isna()
    if mask.any():
        p2 = pd.to_datetime(s[mask], dayfirst=False, errors='coerce')
        p.loc[mask] = p2
    return p

df['Date'] = try_parse_dates(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

def compute_subindex_array(vals, breakpoints):
    vals = pd.to_numeric(vals, errors='coerce').to_numpy(dtype=float)
    out = np.full(vals.shape, np.nan)
    for BPlo, BPhi, Ilo, Ihi in breakpoints:
        mask = (vals >= BPlo) & (vals <= BPhi)
        denom = (BPhi - BPlo) if (BPhi - BPlo) != 0 else 1
        out[mask] = ((Ihi - Ilo) / denom) * (vals[mask] - BPlo) + Ilo
    return out

pm25_bp = [(0,30,0,50),(31,60,51,100),(61,90,101,200),(91,120,201,300),(121,250,301,400),(251,500,401,500)]
pm10_bp = [(0,50,0,50),(51,100,51,100),(101,250,101,200),(251,350,201,300),(351,430,301,400),(431,10000,401,500)]
no2_bp  = [(0,40,0,50),(41,80,51,100),(81,180,101,200),(181,280,201,300),(281,400,301,400),(401,10000,401,500)]
so2_bp  = [(0,40,0,50),(41,80,51,100),(81,380,101,200),(381,800,201,300),(801,1600,301,400),(1601,10000,401,500)]
co_bp   = [(0,1,0,50),(1.01,2,51,100),(2.01,10,101,200),(10.01,17,201,300),(17.01,34,301,400),(34.01,1000,401,500)]
o3_bp   = [(0,50,0,50),(51,100,51,100),(101,168,101,200),(169,208,201,300),(209,748,301,400),(749,10000,401,500)]

si_map = {
    "SI_PM25": ("PM2.5", pm25_bp),
    "SI_PM10": ("PM10", pm10_bp),
    "SI_NO2": ("NO2", no2_bp),
    "SI_SO2": ("SO2", so2_bp),
    "SI_CO": ("CO", co_bp),
    "SI_O3": ("O3", o3_bp)
}

for si_col, (poll_col, bp) in si_map.items():
    if poll_col in df.columns and si_col not in df.columns:
        df[si_col] = compute_subindex_array(df[poll_col], bp)

si_cols = [c for c in df.columns if c.startswith("SI_")]
if len(si_cols) == 0:
    raise RuntimeError("No SI_ columns created. Check pollutant column names.")

df['AQI'] = df[si_cols].apply(pd.to_numeric, errors='coerce').max(axis=1, skipna=True)

def bucket_label(x):
    try:
        x = float(x)
    except:
        return np.nan
    if x <= 50: return "Good"
    if x <= 100: return "Satisfactory"
    if x <= 200: return "Moderate"
    if x <= 300: return "Poor"
    if x <= 400: return "Very Poor"
    return "Severe"

df['AQI_Bucket'] = df['AQI'].apply(bucket_label)

print("AQI_Bucket value counts:\n", df['AQI_Bucket'].value_counts(dropna=True))
print("\nSample rows with Date parsing issues (Date is NaT):")
print(df[df['Date'].isna()].head(10))

# optional: save augmented file
out = r"C:\Users\avi04\Downloads\city_day_descriptive_augmented.csv"
df.to_csv(out, index=False)
print("\nSaved augmented file to:", out)


AQI_Bucket value counts:
 AQI_Bucket
Satisfactory    10086
Moderate         9493
Poor             3178
Very Poor        2995
Good             2857
Severe            922
Name: count, dtype: int64

Sample rows with Date parsing issues (Date is NaT):
Empty DataFrame
Columns: [City, Date, PM2.5, PM10, NO, NO2, NOx, NH3, CO, SO2, O3, Benzene, Toluene, Xylene, Year, Month, SI_PM25, SI_PM10, SI_NO2, SI_SO2, SI_CO, SI_O3, AQI, AQI_Bucket]
Index: []

[0 rows x 24 columns]

Saved augmented file to: C:\Users\avi04\Downloads\city_day_descriptive_augmented.csv
